In [1]:
import numpy as num
import cymetric as cym
import pandas as pd

/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)
/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.material is not yet QA compliant.
  return f(*args, **kwds)
/opt/local/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: QAWarning: pyne.enrichment is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
from cymetric import convenient_interface as cinv

In [3]:
loop_list = ['PWR', 'FBR']
model_list = ['fuelfab', 'mixer', 'neural']
DM_list = ['nodecay', 'decay']
neural_extra_list = ['recipe', 'depletion']

In [4]:
#build calcualation list
cal_list = []
for loop in loop_list[:1]:
    for model in model_list:
        for DM in DM_list:
            if model == 'neural':
                for NN_extra in neural_extra_list:
                    path_to_db_ = loop + '/' + model + '/' + DM + '/'
                    path_to_db_ += NN_extra + '/' + 'cyclus.sqlite'
                    cal_list.append(path_to_db_)
            else:
                path_to_db_ = loop + '/' + model + '/' + DM + '/'
                path_to_db_ += 'cyclus.sqlite'
                cal_list.append(path_to_db_)

In [5]:
for cal in cal_list:
    print(cal[:-13].find("nodecay"))

12
-1
10
-1
11
11
-1
-1


In [6]:
# Fuel enrichement All case
pu_enricht_list = []
for cal in cal_list:
    print(cal)
    db = cym.dbopen(cal)
    fuel_flow = cinv.GetTransactionTimeSeries(db,receiver='PWR')
    fuel_flow_pu = cinv.GetTransactionTimeSeries(db,'All','PWR','pu238','pu239','pu240','pu241','pu242','am241')
    fuel_flow_pu_enrich = fuel_flow_pu/fuel_flow
    fuel_flow_pu_enrich = fuel_flow_pu_enrich.rename(index=str, columns={'Quantity': cal[:-14]})
    fuel_flow_pu_enrich.reset_index(inplace=True)
    pu_enricht_list.append(fuel_flow_pu_enrich)
    filename_ = cal[:-13] + 'Fuel_PU_enricht.dat'
    file = open(filename_,"w")
    file.write('Time Val\n')
    for i in range(len(fuel_flow_pu_enrich)):
        t = float(fuel_flow_pu_enrich.iloc[i,0])/12.
        file.write(str(t) )
        file.write(" ")
        file.write( str(fuel_flow_pu_enrich.iloc[i,1]*100.))
        file.write("\n")
    file.close()
    db.close()

PWR/fuelfab/nodecay/cyclus.sqlite
PWR/fuelfab/decay/cyclus.sqlite
PWR/mixer/nodecay/cyclus.sqlite
PWR/mixer/decay/cyclus.sqlite
PWR/neural/nodecay/recipe/cyclus.sqlite
PWR/neural/nodecay/depletion/cyclus.sqlite
PWR/neural/decay/recipe/cyclus.sqlite
PWR/neural/decay/depletion/cyclus.sqlite


In [7]:
# pu composition IN fuel
for cal in cal_list:
    print(cal)
    db2 = cym.dbopen(cal)
    fuel_flow = cinv.GetTransactionTimeSeries(db2,'All','PWR','pu238','pu239','pu240','pu241','pu242','am241')
    pu_list = ['pu238','pu239','pu240','pu241','pu242']
    if( cal[:-13].find("nodecay") == -1 ):
        pu_list.append('am241')

    fuel_flow_pu = []
    for pu in pu_list:
        fuel_flow_pu_tmp = cinv.GetTransactionTimeSeries(db2,'All','PWR', pu)
        fuel_flow_pu.append( fuel_flow_pu_tmp/fuel_flow)

    filename_ = cal[:-13] + 'Fuel_Pu.composition.dat'
    file = open(filename_,"w")
    nuc = 0
    if( cal[:-13].find("nodecay") == -1 ):
        file.write('Time Pu238 Pu239 Pu240 Pu241 Pu242 Am241\n')
        nuc = 6
    else:
        file.write('Time Pu238 Pu239 Pu240 Pu241 Pu242\n')
        nuc = 5
    for j in range(nuc):
        fuel_flow_pu[j].reset_index(inplace=True)

    for i in range(len(fuel_flow_pu[0])):
        t = float(fuel_flow_pu[0].iloc[i,0])/12.
        file.write( str(t) )
        file.write(" ")
        for j in range(nuc):
            file.write( str(fuel_flow_pu[j].iloc[i,1]*100.))
            file.write(" ")
        file.write("\n")
    file.close()
    db2.close()

PWR/fuelfab/nodecay/cyclus.sqlite
PWR/fuelfab/decay/cyclus.sqlite
PWR/mixer/nodecay/cyclus.sqlite
PWR/mixer/decay/cyclus.sqlite
PWR/neural/nodecay/recipe/cyclus.sqlite
PWR/neural/nodecay/depletion/cyclus.sqlite
PWR/neural/decay/recipe/cyclus.sqlite
PWR/neural/decay/depletion/cyclus.sqlite


In [8]:
# pu amount IN fuel
for cal in cal_list:
    print(cal)
    db2 = cym.dbopen(cal)
    fuel_flow = cinv.GetTransactionTimeSeries(db2,'All','PWR')
    pu_list = ['pu238','pu239','pu240','pu241','pu242']
    if( cal[:-13].find("nodecay") == -1 ):
        pu_list.append('am241')

    fuel_flow_pu = []
    for pu in pu_list:
        fuel_flow_pu_tmp = cinv.GetTransactionTimeSeries(db2,'All','PWR', pu)
        fuel_flow_pu.append( fuel_flow_pu_tmp/fuel_flow)

    filename_ = cal[:-13] + 'Fuel_Pu.amount.dat'
    file = open(filename_,"w")
    nuc = 0
    if( cal[:-13].find("nodecay") == -1 ):
        file.write('Time Pu238 Pu239 Pu240 Pu241 Pu242 Am241\n')
        nuc = 6
    else:
        file.write('Time Pu238 Pu239 Pu240 Pu241 Pu242\n')
        nuc = 5
    for j in range(nuc):
        fuel_flow_pu[j].reset_index(inplace=True)


    for i in range(len(fuel_flow_pu[0])):
        t = float(fuel_flow_pu[0].iloc[i,0])/12.
        file.write( str(t) )
        file.write(" ")
        for j in range(nuc):
            file.write( str(fuel_flow_pu[j].iloc[i,1]*100.))
            file.write(" ")
        file.write("\n")
    file.close()
    db2.close()

PWR/fuelfab/nodecay/cyclus.sqlite
PWR/fuelfab/decay/cyclus.sqlite
PWR/mixer/nodecay/cyclus.sqlite
PWR/mixer/decay/cyclus.sqlite
PWR/neural/nodecay/recipe/cyclus.sqlite
PWR/neural/nodecay/depletion/cyclus.sqlite
PWR/neural/decay/recipe/cyclus.sqlite
PWR/neural/decay/depletion/cyclus.sqlite


In [9]:
# pu composition OUT fuel
for cal in cal_list:
    print(cal)
    db2 = cym.dbopen(cal)
    fuel_flow = cinv.GetTransactionTimeSeries(db2,'PWR','All','pu238','pu239','pu240','pu241','pu242','am241')
    pu_list = ['pu238','pu239','pu240','pu241','pu242']
    if( cal[:-13].find("nodecay") == -1 ):
        pu_list.append('am241')

    fuel_flow_pu = []
    for pu in pu_list:
        fuel_flow_pu_tmp = cinv.GetTransactionTimeSeries(db2,'PWR','All', pu)
        fuel_flow_pu.append( fuel_flow_pu_tmp/fuel_flow)

    filename_ = cal[:-13] + 'Spent_Fuel_Pu.composition.dat'
    file = open(filename_,"w")
    nuc = 0
    if( cal[:-13].find("nodecay") == -1 ):
        file.write('Time Pu238 Pu239 Pu240 Pu241 Pu242 Am241\n')
        nuc = 6
    else:
        file.write('Time Pu238 Pu239 Pu240 Pu241 Pu242\n')
        nuc = 5
    for j in range(nuc):
        fuel_flow_pu[j].reset_index(inplace=True)


    for i in range(len(fuel_flow_pu[0])):
        t = float(fuel_flow_pu[0].iloc[i,0])/12.
        file.write( str(t) )
        file.write(" ")
        for j in range(nuc):
            file.write( str(fuel_flow_pu[j].iloc[i,1]*100.))
            file.write(" ")
        file.write("\n")
    file.close()
    db2.close()

PWR/fuelfab/nodecay/cyclus.sqlite
PWR/fuelfab/decay/cyclus.sqlite
PWR/mixer/nodecay/cyclus.sqlite
PWR/mixer/decay/cyclus.sqlite
PWR/neural/nodecay/recipe/cyclus.sqlite
PWR/neural/nodecay/depletion/cyclus.sqlite
PWR/neural/decay/recipe/cyclus.sqlite
PWR/neural/decay/depletion/cyclus.sqlite


In [10]:
# pu amount OUT fuel
for cal in cal_list:
    print(cal)
    db2 = cym.dbopen(cal)
    fuel_flow = cinv.GetTransactionTimeSeries(db2,'PWR','All')
    pu_list = ['pu238','pu239','pu240','pu241','pu242']
    if( cal[:-13].find("nodecay") == -1 ):
        pu_list.append('am241')

    fuel_flow_pu = []
    for pu in pu_list:
        fuel_flow_pu_tmp = cinv.GetTransactionTimeSeries(db2,'PWR','All', pu)
        fuel_flow_pu.append( fuel_flow_pu_tmp/fuel_flow)

    filename_ = cal[:-13] + 'Spent_Fuel_Pu.amount.dat'
    file = open(filename_,"w")
    nuc = 0
    if( cal[:-13].find("nodecay") == -1 ):
        file.write('Time Pu238 Pu239 Pu240 Pu241 Pu242 Am241\n')
        nuc = 6
    else:
        file.write('Time Pu238 Pu239 Pu240 Pu241 Pu242\n')
        nuc = 5
    for j in range(nuc):
        fuel_flow_pu[j].reset_index(inplace=True)


    for i in range(len(fuel_flow_pu[0])):
        t = float(fuel_flow_pu[0].iloc[i,0])/12.
        file.write( str(t) )
        file.write(" ")
        for j in range(nuc):
            file.write( str(fuel_flow_pu[j].iloc[i,1]*100.))
            file.write(" ")
        file.write("\n")
    file.close()
    db2.close()

PWR/fuelfab/nodecay/cyclus.sqlite
PWR/fuelfab/decay/cyclus.sqlite
PWR/mixer/nodecay/cyclus.sqlite
PWR/mixer/decay/cyclus.sqlite
PWR/neural/nodecay/recipe/cyclus.sqlite
PWR/neural/nodecay/depletion/cyclus.sqlite
PWR/neural/decay/recipe/cyclus.sqlite
PWR/neural/decay/depletion/cyclus.sqlite
